In [ ]:
!pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import json
with open("/content/drive/MyDrive/kaggle.json", 'r') as f:
    json_data = json.load(f)
os.environ['KAGGLE_USERNAME'] = json_data['username']
os.environ['KAGGLE_KEY'] = json_data['key']

In [ ]:
!kaggle competitions download -c jane-street-real-time-market-data-forecasting

100% 11.4G/11.5G [00:47<00:00, 256MB/s]
100% 11.5G/11.5G [00:47<00:00, 260MB/s]


In [ ]:
!unzip jane-street-real-time-market-data-forecasting.zip

Archive:  jane-street-real-time-market-data-forecasting.zip
  inflating: features.csv            
  inflating: kaggle_evaluation/__init__.py  
  inflating: kaggle_evaluation/core/__init__.py  
  inflating: kaggle_evaluation/core/base_gateway.py  
  inflating: kaggle_evaluation/core/generated/__init__.py  
  inflating: kaggle_evaluation/core/generated/kaggle_evaluation_pb2.py  
  inflating: kaggle_evaluation/core/generated/kaggle_evaluation_pb2_grpc.py  
  inflating: kaggle_evaluation/core/kaggle_evaluation.proto  
  inflating: kaggle_evaluation/core/relay.py  
  inflating: kaggle_evaluation/core/templates.py  
  inflating: kaggle_evaluation/jane_street_gateway.py  
  inflating: kaggle_evaluation/jane_street_inference_server.py  
  inflating: lags.parquet/date_id=0/part-0.parquet  
  inflating: responders.csv          
  inflating: sample_submission.csv   
  inflating: test.parquet/date_id=0/part-0.parquet  
  inflating: train.parquet/partition_id=0/part-0.parquet  
  inflating: train.p

In [ ]:
!pip uninstall -y xgboost
!pip install xgboost==1.6.1  # GPUサポートが安定しているバージョンにダウングレード

Found existing installation: xgboost 2.1.2
Uninstalling xgboost-2.1.2:
  Successfully uninstalled xgboost-2.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.9/192.9 MB 5.9 MB/s eta 0:00:00


In [ ]:
!pip install polars
!pip install lightgbm

import numpy as np
import polars as pl
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
import os
import joblib
import gc
from sklearn.model_selection import TimeSeriesSplit
import kaggle_evaluation.jane_street_inference_server

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
# パーケットファイルを読み込み、フィルタリングと必要な処理を行う
df = pl.scan_parquet("train.parquet")
df = df.filter(pl.col("partition_id") == 0).collect()
df = df.with_columns(pl.col('responder_6').shift(1).alias('responder_6_lag1'))
df = df.with_columns(pl.col('responder_6').shift(2).alias('responder_6_lag2'))
df = df.slice(0, len(df) - 1)

# 目的変数と特徴量の定義
TARGET = 'responder_6'
FEAT_COLS = [f"feature_{i:02d}" for i in range(79)]
RES_FEAT_COLS = [f"responder_{i}" for i in range(9)]
LAG_RES_FEAT_COLS = ['responder_6_lag1','responder_6_lag2']

# 列を結合して、'responder_6' を除く
#COLS = FEAT_COLS + RES_FEAT_COLS + LAG_RES_FEAT_COLS
COLS = [x for x in FEAT_COLS if x != TARGET]

df[COLS].tail()  # 確認のため出力

feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,…,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78
f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
null,null,null,null,null,-0.020184,0.192278,0.065021,-0.347026,64,6,376,-0.064175,-0.367097,0.000438,-0.385349,-0.47505,-0.409575,-1.361472,1.181357,-0.183469,null,0.888973,0.125289,-0.920569,0.531765,null,null,1.277653,-0.526081,-0.449444,null,0.193946,0.779264,0.203639,0.254882,0.829842,…,1.082042,-0.679178,-0.092983,-0.016576,0.48,0.555043,-0.33077,-0.200343,-0.248864,-0.532577,-1.591908,-0.252836,0.322998,-0.567509,0.923749,-0.38469,0.427977,-0.078065,-0.245995,0.300441,0.770848,-0.301014,0.458725,0.103956,0.826847,-0.136092,-0.192107,-0.044648,0.10291,-0.370744,0.071086,0.566694,0.403683,-0.212804,-0.1807,0.245105,0.372971
null,null,null,null,null,-0.028087,0.287438,0.118074,-0.644495,4,3,11,3.419741,0.997164,1.706084,-0.318914,-0.461119,-0.294792,1.929656,1.213011,1.16521,null,3.431933,0.792091,-0.961323,0.197048,null,null,0.258624,-0.538528,-0.306105,null,0.12894,-0.022014,0.749626,0.547256,1.428185,…,-0.712776,1.209243,-0.337086,-0.205467,-2.51014,-1.281748,-0.383061,0.99984,1.178574,-1.942191,0.445592,-1.414607,-1.362255,-0.760373,-0.199131,-2.412528,-1.033167,-2.285751,0.406769,0.300441,-0.072687,0.114416,0.200983,2.923207,1.137057,5.132564,0.726686,1.304514,3.247437,0.631181,2.437627,-0.341539,-0.360914,-0.267972,-0.253485,-0.147347,-0.166964
null,null,null,null,null,-0.022584,0.442352,0.140746,-0.571057,81,2,534,0.385333,0.732816,0.155896,0.202543,0.930962,0.721754,-0.865098,0.108584,-0.640984,null,-0.876378,-1.044793,-1.282868,-0.355638,null,null,0.776003,-0.126703,0.290924,null,-0.123493,0.995562,-0.466438,-0.349856,-0.892137,…,1.439713,2.302129,2.046488,0.117269,-0.105732,0.343813,-0.132489,0.081151,0.247478,1.210106,0.012775,-0.743068,0.18959,0.866432,0.588312,-0.599171,0.269633,0.248799,0.33105,0.300441,1.796084,2.408884,1.916744,0.658252,0.340445,0.652627,-0.216905,-0.185448,0.521877,1.35906,0.494098,-0.20115,-0.289628,-0.476703,-0.373956,-0.356012,-0.35281
null,null,null,null,null,-0.024804,0.420692,0.136259,-0.809642,11,7,76,-0.200244,-0.409379,-0.185599,0.234811,0.160243,0.110508,-0.959411,1.392733,2.161604,null,-0.219788,-0.040412,-1.940056,-0.637217,null,null,0.481473,-0.211353,0.417266,null,-0.111355,0.543678,0.090486,-0.336618,1.615937,…,0.210434,-0.663065,-0.184241,0.453498,-0.814056,0.076486,0.384369,0.880885,0.198965,1.093185,1.355752,0.86659,-0.543416,-0.557341,1.018626,-0.53542,-0.299109,-0.135551,0.302495,0.300441,1.178458,1.963925,2.115989,2.300967,1.213972,-0.375353,-0.315288,-0.377317,-0.018499,-0.303676,0.042694,-0.304563,-0.274469,-0.339679,-0.301338,-0.323033,-0.239716
null,null,null,null,null,-0.016138,0.303561,0.14997,-0.727993,42,5,150,0.089536,-0.318639,0.229737,-0.359855,-0.135196,-0.361641,-1.061814,1.642337,0.124642,null,-0.523115,-0.88477,-1.407083,-0.564234,null,null,1.599255,-0.414362,-0.21513,null,-0.042788,-0.010577,0.227581,-0.202145,-0.719912,…,1.309015,0.680563,0.854657,1.16664,1.67775

In [ ]:
# # インストール
# !pip install kaggle
# import os
# import json
# with open("/content/drive/MyDrive/kaggle.json", 'r') as f:
#     json_data = json.load(f)
# os.environ['KAGGLE_USERNAME'] = json_data['username']
# os.environ['KAGGLE_KEY'] = json_data['key']
# !kaggle competitions download -c jane-street-real-time-market-data-forecasting
# !unzip jane-street-real-time-market-data-forecasting.zip
# !pip uninstall -y xgboost
# !pip install xgboost==1.6.1  # GPUサポートが安定しているバージョンにダウングレード
# !pip install polars
# !pip install lightgbm

# import numpy as np
# import polars as pl
# import pandas as pd
# import lightgbm as lgb
# import xgboost as xgb
# import os
# import joblib
# import gc
# from sklearn.model_selection import TimeSeriesSplit
# import kaggle_evaluation.jane_street_inference_server

# データの読み込み
df = pl.scan_parquet("train.parquet")
df = df.with_columns(pl.col('responder_6').shift(1).alias('responder_6_lag1'))
df = df.with_columns(pl.col('responder_6').shift(2).alias('responder_6_lag2'))

# 目的変数と特徴量の定義
TARGET = 'responder_6'
FEAT_COLS = [f"feature_{i:02d}" for i in range(79)]
RES_FEAT_COLS = [f"responder_{i}" for i in range(9)]
LAG_RES_FEAT_COLS = ['responder_6_lag1','responder_6_lag2']
COLS = FEAT_COLS + RES_FEAT_COLS + LAG_RES_FEAT_COLS
COLS = [x for x in COLS if x != TARGET]

# フィルタリングを通してParquet ファイルからデータをロード
def load_data(date_id_range=None, time_id_range=None, columns=None, return_type='pl'):
    # parquet形式 : テキスト形式では閲覧できない形式であるがcsvファイルより軽量化され、分析を行う際も高速になる
    data = pl.scan_parquet("train.parquet")
    data = df.with_columns(pl.col('responder_6').shift(1).alias('responder_6_lag1'))
    data = df.with_columns(pl.col('responder_6').shift(2).alias('responder_6_lag2'))

    # date_id_rangeが指定されていれば、開始日と終了日を基にdate_idカラムでデータをフィルタリング
    if date_id_range:
        start_date, end_date = date_id_range
        data = data.filter((pl.col("date_id") >= start_date) & (pl.col("date_id") <= end_date))

    # time_id_rangeが指定されている場合は、開始時間と終了時間でtime_idカラムをフィルタリング
    if time_id_range:
        start_time, end_time = time_id_range
        data = data.filter((pl.col("time_id") >= start_time) & (pl.col("time_id") <= end_time))

    # columnsが指定されている場合、そのカラムだけを選択してデータを絞り込み
    if columns:
        data = data.select(columns)

    # お好みで(polarsの方が早い)
    if return_type == 'pd':
        return data.collect().to_pandas()
    else:
        return data.collect()

# 重み付きR2スコアを出力
def calculate_r2(y_true, y_pred, weights):
    numerator = np.sum(weights * (y_true - y_pred) ** 2)
    denominator = np.sum(weights * (y_true ** 2))
    return 1 - (numerator / denominator)

# XGBoostのカスタム損失関数
def weighted_squared_error(preds, dtrain):
    labels = dtrain.get_label()
    weights = dtrain.get_weight()
    errors = preds - labels
    gradient = 2 * weights * errors
    hessian = 2 * weights
    return gradient, hessian

# RMSEの計算
def weighted_rmse(preds, dtrain):
    labels = dtrain.get_label()
    weights = dtrain.get_weight()
    weighted_mse = np.sum(weights * (preds - labels) ** 2) / np.sum(weights)
    return "weighted_rmse", np.sqrt(weighted_mse)

# 評価関数
def evaluate_model(model, test_data):
    y_pred = model.predict(test_data[COLS])
    y_true = test_data[TARGET].to_numpy()
    weights = test_data['weight'].to_numpy()
    r2_score = calculate_r2(y_true, y_pred, weights)
    print(f"Sample weighted zero-mean R-squared score (R2) on test data: {r2_score}")

# 複数のモデルを一つのオブジェクトにまとめることで、後に予測時に平均化などを行うための土台を作る
class ModelGroup:
    def __init__(self):
        self.models = []

    # モデルを追加
    def add_model(self, model):
        self.models.append(model)

    # 学習した各モデルから予測値を作成
    def predict(self, test_data):
        preds = []
        for model in self.models:
            if isinstance(model, lgb.Booster):
                pred = model.predict(test_data[FEAT_COLS])
            elif isinstance(model, xgb.Booster):
                pred = model.predict(xgb.DMatrix(test_data[FEAT_COLS]))
            elif hasattr(model, 'predict'):
                pred = model.predict(test_data[FEAT_COLS]) # For PyTorch or other model
            else:
                raise ValueError("Unsupported model type")
            preds.append(pred)

        return np.mean(preds, axis=0)

    @classmethod
    def load(cls, file_path):
        return joblib.load(file_path)

# XGboostモデルの定義
def train_xgb_kfold(total_days=1498, n_splits=5, save_models=True, num_boost_round=500):
    # 目的変数と特徴量の定義
    TARGET = 'responder_6'
    FEAT_COLS = [f"feature_{i:02d}" for i in range(79)]
    RES_FEAT_COLS = [f"responder_{i}" for i in range(9)]
    LAG_RES_FEAT_COLS = ['responder_6_lag1','responder_6_lag2']
    COLS = FEAT_COLS + RES_FEAT_COLS + LAG_RES_FEAT_COLS
    COLS = [x for x in COLS if x != TARGET]

    # 時系列の分割
    tscv = TimeSeriesSplit(n_splits=n_splits, max_train_size=50000) #メモリの都合によりサイズを調整

    # インスタンス作成
    model_group = ModelGroup()
    model = None

    # 分割データの中で訓練データと評価データに分割
    for fold_idx, (train_index, valid_index) in enumerate(tscv.split(range(total_days))):
        sampled_train_index = train_index[:int(0.8 * len(train_index))]
        train_range = (sampled_train_index[0], sampled_train_index[-1])
        valid_range = (valid_index[0], valid_index[-1])

        # 評価データの作成
        valid_data = load_data(date_id_range=valid_range, columns=["date_id", "weight"] + COLS + [TARGET], return_type='pl')
        valid_data = valid_data.with_columns([pl.col(c).cast(pl.Float32) for c in COLS + [TARGET, "weight"]]) #データをFloat32にキャスト

        # 検証データの重みとラベルを xgboost.DMatrix 形式に変換
        valid_weight = valid_data['weight'].to_numpy()
        dvalid = xgb.DMatrix(valid_data.select(COLS).to_pandas(), label=valid_data[TARGET].to_pandas(), weight=valid_weight)

        # バッチサイズの指定
        batch_size = 500 if fold_idx == n_splits - 1 else 2000  # 最後のfoldは小さなバッチサイズ

        # 学習データの作成
        for train_idx in range(train_range[0], train_range[1], batch_size):
            end_idx = min(train_idx + batch_size - 1, train_range[1])
            partial_train_data = load_data(date_id_range=(train_idx, end_idx), columns=["date_id", "weight"] + COLS + [TARGET], return_type='pl')
            partial_train_data = partial_train_data.with_columns([pl.col(c).cast(pl.Float32) for c in COLS + [TARGET, "weight"]])
            train_weight = partial_train_data['weight'].to_numpy()
            dtrain = xgb.DMatrix(partial_train_data.select(COLS).to_pandas(), label=partial_train_data[TARGET].to_pandas(), weight=train_weight)

            # パラメータ設定
            XGB_PARAMS = {
                'objective': 'reg:squarederror',
                'learning_rate': 0.03,
                'max_depth': 3,
                'min_child_weight': 3,
                'subsample': 0.7,
                'colsample_bytree': 0.7,
                'random_state': 42,
                'tree_method': 'gpu_hist',
            }

            model = xgb.train(
                XGB_PARAMS,
                dtrain,
                num_boost_round=num_boost_round,
                xgb_model=model,
                evals=[(dtrain, 'train'), (dvalid, 'valid')],
                early_stopping_rounds=50,
                verbose_eval=50,
                obj=weighted_squared_error,
                custom_metric=weighted_rmse
            )

            # 余計なものは削除
            del partial_train_data, dtrain, train_weight
            gc.collect()

        # R2を計算
        y_valid_pred = model.predict(dvalid)
        r2_score = calculate_r2(valid_data[TARGET].to_numpy(), y_valid_pred, valid_weight)
        print(f"Fold {fold_idx} validation R2 score: {r2_score}")

        model_group.add_model(model)

        if save_models:
            joblib.dump(model, f"xgb_model_group_{fold_idx}.pkl")
            print(f"Saved the model for fold {fold_idx} to xgb_model_group_{fold_idx}.pkl")

        del dvalid, valid_data, valid_weight, y_valid_pred
        gc.collect()

    return model_group



In [ ]:
# 実行例
total_days = 1498
xgb_models = train_xgb_kfold(total_days=total_days, n_splits=5)

[0]	train-rmse:0.95824	train-weighted_rmse:0.95824	valid-rmse:1.00505	valid-weighted_rmse:1.00505
[50]	train-rmse:0.60902	train-weighted_rmse:0.60902	valid-rmse:0.60518	valid-weighted_rmse:0.60518
[100]	train-rmse:0.52796	train-weighted_rmse:0.52796	valid-rmse:0.50113	valid-weighted_rmse:0.50113
[150]	train-rmse:0.49244	train-weighted_rmse:0.49244	valid-rmse:0.45874	valid-weighted_rmse:0.45874
[200]	train-rmse:0.47021	train-weighted_rmse:0.47020	valid-rmse:0.43298	valid-weighted_rmse:0.43298
[250]	train-rmse:0.45523	train-weighted_rmse:0.45523	valid-rmse:0.41735	valid-weighted_rmse:0.41735
[300]	train-rmse:0.44418	train-weighted_rmse:0.44418	valid-rmse:0.40467	valid-weighted_rmse:0.40467
[350]	train-rmse:0.43591	train-weighted_rmse:0.43591	valid-rmse:0.39486	valid-weighted_rmse:0.39486
[400]	train-rmse:0.42936	train-weighted_rmse:0.42936	valid-rmse:0.38706	valid-weighted_rmse:0.38706
[450]	train-rmse:0.42437	train-weighted_rmse:0.42437	valid-rmse:0.38190	valid-weighted_rmse:0.38190
[49

In [ ]:
#モデルファイルをドライブから持ってくる
import joblib
class ModelGroup:
    def __init__(self):
        self.models = []

    def add_model(self, model):
        """Add a trained model to the group."""
        self.models.append(model)

    def predict(self, test_data):
        """Make predictions using all models in the group and return the average."""
        preds = []
        for model in self.models:
            if isinstance(model, lgb.Booster):
                pred = model.predict(test_data[FEAT_COLS])
            elif isinstance(model, xgb.Booster):
                pred = model.predict(xgb.DMatrix(test_data[FEAT_COLS]))
            elif hasattr(model, 'predict'):  # For PyTorch or other models
                pred = model.predict(test_data[FEAT_COLS])
            else:
                raise ValueError("Unsupported model type")

            preds.append(pred)

        # Average the predictions from all models
        avg_pred = np.mean(preds, axis=0)
        return avg_pred

    @classmethod
    def load(cls, file_path):
        """Load a model group from a file."""
        model_group = joblib.load(file_path)
        return model_group

lgb_model = ModelGroup.load("drive/MyDrive/lgb_model_group_v3.pkl")


In [ ]:
#lags_ というグローバル変数を宣言し、型は Polars データフレームまたは None として定義
lags_ : pl.DataFrame | None = None
def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    # lags_ をグローバル変数として使用し、lags が渡されている場合は、lags_ に代入
    global lags_
    if lags is not None:
        lags_ = lags
    # test データフレームから row_id 列を選択し、responder_6 列を0で仮埋め
    predictions = test.select(
        'row_id',
        pl.lit(0.0).alias('responder_6'),
    )

    # 予測値出力
    feat = test[FEAT_COLS].to_pandas()
    pred = lgb_model.predict(feat)

    # 予測値の出力
    predictions = predictions.with_columns(pl.Series('responder_6', pred.ravel()))
    print(predictions)

    # The predict function must return a DataFrame
    assert isinstance(predictions, pl.DataFrame | pd.DataFrame)
    # with columns 'row_id', 'responder_6'
    assert list(predictions.columns) == ['row_id', 'responder_6']
    # and as many rows as the test data.
    assert len(predictions) == len(test)

    return predictions

inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            'test.parquet',
            'lags.parquet',
        )
    )

shape: (39, 2)
┌────────┬─────────────┐
│ row_id ┆ responder_6 │
│ ---    ┆ ---         │
│ i64    ┆ f64         │
╞════════╪═════════════╡
│ 0      ┆ -0.010348   │
│ 1      ┆ -0.010348   │
│ 2      ┆ -0.010348   │
│ 3      ┆ -0.010348   │
│ 4      ┆ -0.010348   │
│ …      ┆ …           │
│ 34     ┆ -0.010348   │
│ 35     ┆ -0.010348   │
│ 36     ┆ -0.011446   │
│ 37     ┆ -0.010348   │
│ 38     ┆ -0.010348   │
└────────┴─────────────┘


In [ ]:
# インストール
!pip install kaggle
import os
import json
!kaggle competitions download -c jane-street-real-time-market-data-forecasting
!unzip jane-street-real-time-market-data-forecasting.zip
import numpy as np
import polars as pl
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
import os
import joblib
import gc
from sklearn.model_selection import TimeSeriesSplit
import kaggle_evaluation.jane_street_inference_server

# 目的変数と特徴量の定義
TARGET = 'responder_6'
FEAT_COLS = [f"feature_{i:02d}" for i in range(79)]

#モデルファイルをドライブから持ってくる
import joblib
class ModelGroup:
    def __init__(self):
        self.models = []

    def add_model(self, model):
        """Add a trained model to the group."""
        self.models.append(model)

    def predict(self, test_data):
        """Make predictions using all models in the group and return the average."""
        preds = []
        for model in self.models:
            if isinstance(model, lgb.Booster):
                pred = model.predict(test_data[FEAT_COLS])
            elif isinstance(model, xgb.Booster):
                pred = model.predict(xgb.DMatrix(test_data[FEAT_COLS]))
            elif hasattr(model, 'predict'):  # For PyTorch or other models
                pred = model.predict(test_data[FEAT_COLS])
            else:
                raise ValueError("Unsupported model type")

            preds.append(pred)

        # Average the predictions from all models
        avg_pred = np.mean(preds, axis=0)
        return avg_pred

    @classmethod
    def load(cls, file_path):
        """Load a model group from a file."""
        model_group = joblib.load(file_path)
        return model_group

lgb_model = ModelGroup.load("drive/MyDrive/lgb_model_group_v3.pkl")

#lags_ というグローバル変数を宣言し、型は Polars データフレームまたは None として定義
lags_ : pl.DataFrame | None = None


def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    # lags_ をグローバル変数として使用し、lags が渡されている場合は、lags_ に代入
    global lags_
    if lags is not None:
        lags_ = lags
    # test データフレームから row_id 列を選択し、responder_6 列を0で仮埋め
    predictions = test.select(
        'row_id',
        pl.lit(0.0).alias('responder_6'),
    )

    # 予測値出力
    feat = test[FEAT_COLS].to_pandas()
    pred = lgb_model.predict(feat)

    # 予測値の出力
    predictions = predictions.with_columns(pl.Series('responder_6', pred.ravel()))
    print(predictions)

    # The predict function must return a DataFrame
    assert isinstance(predictions, pl.DataFrame | pd.DataFrame)
    # with columns 'row_id', 'responder_6'
    assert list(predictions.columns) == ['row_id', 'responder_6']
    # and as many rows as the test data.
    assert len(predictions) == len(test)

    return predictions

inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '../input/jane-street-real-time-market-data-forecasting/test.parquet',
            '../input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )

In [ ]:
# インストール
!pip install kaggle
!pip install kaggle_evaluation
!pip install kaggle_evaluation.jane_street_inference_server
import os
import json
!kaggle competitions download -c jane-street-real-time-market-data-forecasting
!unzip jane-street-real-time-market-data-forecasting.zip
import numpy as np
import polars as pl
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
import os
import joblib
import gc
from sklearn.model_selection import TimeSeriesSplit
import kaggle_evaluation.jane_street_inference_server

# 目的変数と特徴量の定義
TARGET = 'responder_6'
FEAT_COLS = [f"feature_{i:02d}" for i in range(79)]

#モデルファイルをドライブから持ってくる
import joblib
class ModelGroup:
    def __init__(self):
        self.models = []

    def add_model(self, model):
        """Add a trained model to the group."""
        self.models.append(model)

    def predict(self, test_data):
        """Make predictions using all models in the group and return the average."""
        preds = []
        for model in self.models:
            if isinstance(model, lgb.Booster):
                pred = model.predict(test_data[FEAT_COLS])
            elif isinstance(model, xgb.Booster):
                pred = model.predict(xgb.DMatrix(test_data[FEAT_COLS]))
            elif hasattr(model, 'predict'):  # For PyTorch or other models
                pred = model.predict(test_data[FEAT_COLS])
            else:
                raise ValueError("Unsupported model type")

            preds.append(pred)

        # Average the predictions from all models
        avg_pred = np.mean(preds, axis=0)
        return avg_pred

    @classmethod
    def load(cls, file_path):
        """Load a model group from a file."""
        model_group = joblib.load(file_path)
        return model_group

lgb_model = ModelGroup.load("/kaggle/input/lgb_model_group_v3/other/default/1/lgb_model_group_v3.pkl")

#lags_ というグローバル変数を宣言し、型は Polars データフレームまたは None として定義
lags_ : pl.DataFrame | None = None


def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    # lags_ をグローバル変数として使用し、lags が渡されている場合は、lags_ に代入
    global lags_
    if lags is not None:
        lags_ = lags
    # test データフレームから row_id 列を選択し、responder_6 列を0で仮埋め
    predictions = test.select(
        'row_id',
        pl.lit(0.0).alias('responder_6'),
    )

    # 予測値出力
    feat = test[FEAT_COLS].to_pandas()
    pred = lgb_model.predict(feat)

    # 予測値の出力
    predictions = predictions.with_columns(pl.Series('responder_6', pred.ravel()))
    print(predictions)

    # The predict function must return a DataFrame
    assert isinstance(predictions, pl.DataFrame | pd.DataFrame)
    # with columns 'row_id', 'responder_6'
    assert list(predictions.columns) == ['row_id', 'responder_6']
    # and as many rows as the test data.
    assert len(predictions) == len(test)

    return predictions

inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '../input/jane-street-real-time-market-data-forecasting/test.parquet',
            '../input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )

In [ ]:
# インストール
!pip install kaggle
import os
import json
# with open("/content/drive/MyDrive/kaggle.json", 'r') as f:
#     json_data = json.load(f)
# os.environ['KAGGLE_USERNAME'] = json_data['username']
# os.environ['KAGGLE_KEY'] = json_data['key']
!kaggle competitions download -c jane-street-real-time-market-data-forecasting
!unzip jane-street-real-time-market-data-forecasting.zip


import numpy as np
import polars as pl
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
import os
import joblib
import gc
from sklearn.model_selection import TimeSeriesSplit
import kaggle_evaluation.jane_street_inference_server

# 目的変数と特徴量の定義
TARGET = 'responder_6'
FEAT_COLS = [f"feature_{i:02d}" for i in range(79)]

# フィルタリングを通してParquet ファイルからデータをロード
def load_data(date_id_range=None, time_id_range=None, columns=None, return_type='pl'):
    # parquet形式 : テキスト形式では閲覧できない形式であるがcsvファイルより軽量化され、分析を行う際も高速になる
    data_dir = '../input/jane-street-real-time-market-data-forecasting'
    # Load data using Polars lazy loading (scan_parquet)
    data = pl.scan_parquet(f"{data_dir}/train.parquet")

    # date_id_rangeが指定されていれば、開始日と終了日を基にdate_idカラムでデータをフィルタリング
    if date_id_range:
        start_date, end_date = date_id_range
        data = data.filter((pl.col("date_id") >= start_date) & (pl.col("date_id") <= end_date))

    # time_id_rangeが指定されている場合は、開始時間と終了時間でtime_idカラムをフィルタリング
    if time_id_range:
        start_time, end_time = time_id_range
        data = data.filter((pl.col("time_id") >= start_time) & (pl.col("time_id") <= end_time))

    # columnsが指定されている場合、そのカラムだけを選択してデータを絞り込み
    if columns:
        data = data.select(columns)

    # お好みで(polarsの方が早い)
    if return_type == 'pd':
        return data.collect().to_pandas()
    else:
        return data.collect()

# 重み付きR2スコアを出力
def calculate_r2(y_true, y_pred, weights):
    numerator = np.sum(weights * (y_true - y_pred) ** 2)
    denominator = np.sum(weights * (y_true ** 2))
    return 1 - (numerator / denominator)

# XGBoostのカスタム損失関数
def weighted_squared_error(preds, dtrain):
    labels = dtrain.get_label()
    weights = dtrain.get_weight()
    errors = preds - labels
    gradient = 2 * weights * errors
    hessian = 2 * weights
    return gradient, hessian

# RMSEの計算
def weighted_rmse(preds, dtrain):
    labels = dtrain.get_label()
    weights = dtrain.get_weight()
    weighted_mse = np.sum(weights * (preds - labels) ** 2) / np.sum(weights)
    return "weighted_rmse", np.sqrt(weighted_mse)

# 評価関数
def evaluate_model(model, test_data):
    y_pred = model.predict(test_data[FEAT_COLS])
    y_true = test_data[TARGET].to_numpy()
    weights = test_data['weight'].to_numpy()
    r2_score = calculate_r2(y_true, y_pred, weights)
    print(f"Sample weighted zero-mean R-squared score (R2) on test data: {r2_score}")

# 複数のモデルを一つのオブジェクトにまとめることで、後に予測時に平均化などを行うための土台を作る
class ModelGroup:
    def __init__(self):
        self.models = []

    # モデルを追加
    def add_model(self, model):
        self.models.append(model)

    # 学習した各モデルから予測値を作成
    def predict(self, test_data):
        preds = []
        for model in self.models:
            if isinstance(model, lgb.Booster):
                pred = model.predict(test_data[FEAT_COLS])
            elif isinstance(model, xgb.Booster):
                pred = model.predict(xgb.DMatrix(test_data[FEAT_COLS]))
            elif hasattr(model, 'predict'):
                pred = model.predict(test_data[FEAT_COLS]) # For PyTorch or other model
            else:
                raise ValueError("Unsupported model type")
            preds.append(pred)

        return np.mean(preds, axis=0)

    @classmethod
    def load(cls, file_path):
        return joblib.load(file_path)

# XGboostモデルの定義
def train_xgb_kfold(total_days=1498, n_splits=5, save_models=True, num_boost_round=50):
    # 目的変数と特徴量の定義
    TARGET = 'responder_6'
    FEAT_COLS = [f"feature_{i:02d}" for i in range(79)]

    # 時系列の分割
    tscv = TimeSeriesSplit(n_splits=n_splits, max_train_size=100) #メモリの都合によりサイズを調整

    # インスタンス作成
    model_group = ModelGroup()
    model = None

    # 分割データの中で訓練データと評価データに分割
    for fold_idx, (train_index, valid_index) in enumerate(tscv.split(range(total_days))):
        sampled_train_index = train_index[-int(0.8 * len(train_index)):]
        train_range = (sampled_train_index[0], sampled_train_index[-1])
        valid_range = (valid_index[0], valid_index[-1])

        # 評価データの作成
        valid_data = load_data(date_id_range=valid_range, columns=["date_id", "weight"] + FEAT_COLS + [TARGET], return_type='pl')
        valid_data = valid_data.with_columns([pl.col(c).cast(pl.Float32) for c in FEAT_COLS + [TARGET, "weight"]]) #データをFloat32にキャスト

        # 検証データの重みとラベルを xgboost.DMatrix 形式に変換
        valid_weight = valid_data['weight'].to_numpy()
        dvalid = xgb.DMatrix(valid_data.select(FEAT_COLS).to_pandas(), label=valid_data[TARGET].to_pandas(), weight=valid_weight)

        # バッチサイズの指定
        batch_size = 1000 if fold_idx == n_splits - 1 else 3000  # 最後のfoldは小さなバッチサイズ

        # 学習データの作成
        for train_idx in range(train_range[0], train_range[1], batch_size):
            end_idx = min(train_idx + batch_size - 1, train_range[1])
            partial_train_data = load_data(date_id_range=(train_idx, end_idx), columns=["date_id", "weight"] + FEAT_COLS + [TARGET], return_type='pl')
            partial_train_data = partial_train_data.with_columns([pl.col(c).cast(pl.Float32) for c in FEAT_COLS + [TARGET, "weight"]])
            train_weight = partial_train_data['weight'].to_numpy()
            dtrain = xgb.DMatrix(partial_train_data.select(FEAT_COLS).to_pandas(), label=partial_train_data[TARGET].to_pandas(), weight=train_weight)

            # パラメータ設定
            XGB_PARAMS = {
                'objective': 'reg:squarederror',
                'learning_rate': 0.05,
                'max_depth': 1,
                'min_child_weight': 3,
                'subsample': 0.7,
                'colsample_bytree': 0.7,
                'random_state': 42,
                'tree_method': 'hist',
            }

            model = xgb.train(
                XGB_PARAMS,
                dtrain,
                num_boost_round=num_boost_round,
                xgb_model=model,
                evals=[(dtrain, 'train'), (dvalid, 'valid')],
                early_stopping_rounds=10,
                verbose_eval=50,
                obj=weighted_squared_error,
                custom_metric=weighted_rmse
            )

            # 余計なものは削除
            del partial_train_data, dtrain, train_weight
            gc.collect()

        # R2を計算
        y_valid_pred = model.predict(dvalid)
        r2_score = calculate_r2(valid_data[TARGET].to_numpy(), y_valid_pred, valid_weight)
        print(f"Fold {fold_idx} validation R2 score: {r2_score}")

        model_group.add_model(model)

        if save_models:
            joblib.dump(model, f"xgb_model_group_{fold_idx}.pkl")
            print(f"Saved the model for fold {fold_idx} to xgb_model_group_{fold_idx}.pkl")

        del dvalid, valid_data, valid_weight, y_valid_pred
        gc.collect()

    return model_group

# 実行例
total_days = 199
xgb_models = train_xgb_kfold(total_days=total_days, n_splits=5)

#lags_ というグローバル変数を宣言し、型は Polars データフレームまたは None として定義
lags_ : pl.DataFrame | None = None


def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    # lags_ をグローバル変数として使用し、lags が渡されている場合は、lags_ に代入
    global lags_
    if lags is not None:
        lags_ = lags
    # test データフレームから row_id 列を選択し、responder_6 列を0で仮埋め
    predictions = test.select(
        'row_id',
        pl.lit(0.0).alias('responder_6'),
    )

    # 予測値出力
    feat = test[FEAT_COLS].to_pandas()
    pred = xgb_models.predict(feat)

    # 予測値の出力
    predictions = predictions.with_columns(pl.Series('responder_6', pred.ravel()))
    print(predictions)

    # The predict function must return a DataFrame
    assert isinstance(predictions, pl.DataFrame | pd.DataFrame)
    # with columns 'row_id', 'responder_6'
    assert list(predictions.columns) == ['row_id', 'responder_6']
    # and as many rows as the test data.
    assert len(predictions) == len(test)

    return predictions

inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '../input/jane-street-real-time-market-data-forecasting/test.parquet',
            '../input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )